# チャットボットを構築する

:::note

このチュートリアルでは以前、[RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 抽象化を使用していました。このバージョンのドキュメントは [v0.2 ドキュメント](https://python.langchain.com/v0.2/docs/tutorials/chatbot/) でご覧いただけます。

LangChain v0.3 リリース以降、LangChain ユーザーは [LangGraph 永続化](https://langchain-ai.github.io/langgraph/concepts/persistence/) を活用して、新しい LangChain アプリケーションに `memory` を組み込むことをお勧めします。

既にコードで `RunnableWithMessageHistory` または `BaseChatMessageHistory` を使用している場合は、変更する必要はありません。この機能はシンプルなチャットアプリケーションでは機能するため、近い将来に廃止する予定はありません。また、`RunnableWithMessageHistory` を使用するコードは引き続き期待どおりに動作します。

詳細については、[LangGraph Memory への移行方法](/docs/versions/migrating_memory/) をご覧ください。

:::

## 概要

LLM を活用したチャットボットの設計と実装方法を例を挙げて説明します。

このチャットボットは、[チャットモデル](/docs/concepts/chat_models) との会話が可能で、過去のやり取りを記憶します。
構築するチャットボットは、会話を行うために言語モデルのみを使用する点に注意してください。
他にも関連概念がいくつかありますので、ぜひご覧ください。

- [会話型 RAG](/docs/tutorials/qa_chat_history): 外部データソースを介したチャットボットエクスペリエンスを実現する
- [エージェント](/docs/tutorials/agents): アクションを実行できるチャットボットを構築する

このチュートリアルでは、これら 2 つのより高度なトピックに役立つ基本事項を取り上げますが、必要に応じてそちらに直接進んでいただいても構いません。

## セットアップ

### Jupyter Notebook

このガイド（およびドキュメント内の他のほとんどのガイド）では、[Jupyter Notebook](https://jupyter.org/) を使用し、読者も同様にJupyter Notebookを使用することを前提としています。Jupyter Notebookは、LLMシステムの使い方を学ぶのに最適です。なぜなら、予期しない出力やAPIのダウンなど、問題が発生することがよくあるからです。そのため、インタラクティブな環境でガイドを進めることで、より深く理解することができます。

このチュートリアルやその他のチュートリアルは、Jupyter Notebookで実行するのが最も便利です。インストール方法については、[こちら](https://jupyter.org/install) をご覧ください。

### インストール

このチュートリアルでは、`langchain-core` と `langgraph` が必要です。このガイドでは `langgraph >= 0.2.28` が必要です。

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain-core langgraph>0.2.27</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain-core langgraph>0.2.27 -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



詳細については、[インストールガイド](/docs/how_to/installation)をご覧ください。

### LangSmith

LangChainで構築するアプリケーションの多くは、複数のステップでLLM呼び出しを複数回実行します。
これらのアプリケーションが複雑になるにつれて、チェーンまたはエージェント内で何が起こっているかを正確に検査することが重要になります。
これを行うには、[LangSmith](https://smith.langchain.com)を使用するのが最適です。

上記のリンクでサインアップしたら、環境変数を設定してトレースのログ記録を開始してください:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

または、ノートブックの場合は次のように設定できます:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

## クイックスタート

まずは、言語モデル単体の使い方を学びましょう。LangChainは様々な言語モデルをサポートしており、相互に活用できます。以下からご希望の言語モデルを選択してください。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />


In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

まずはモデルを直接使ってみましょう。`ChatModel`はLangChainの「Runnable」のインスタンスであり、操作するための標準インターフェースを公開しています。モデルを呼び出すには、`.invoke`メソッドにメッセージのリストを渡します。

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-5211544f-da9f-4325-8b8e-b3d92b2fc71a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

モデル自体には状態の概念はありません。例えば、次のような質問をしてみましょう:

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal information about users unless it has been shared with me in the course of our conversation. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 11, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2d13a18-7022-4784-b54f-f85c097d1075-0', usage_metadata={'input_tokens': 11, 'output_tokens': 34, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

[LangSmith トレース](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r) の例を見てみましょう。

前の会話の展開が考慮されておらず、質問に答えられないことがわかります。
これはチャットボットのエクスペリエンスを著しく損なうものです。

この問題を回避するには、[会話履歴](/docs/concepts/chat_history) 全体をモデルに渡す必要があります。そうするとどうなるか見てみましょう。

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 33, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-34bcccb3-446e-42f2-b1de-52c09936c02c-0', usage_metadata={'input_tokens': 33, 'output_tokens': 14, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

これで、良好な応答が得られていることがわかります！

これが、チャットボットが会話形式でやり取りする能力を支える基本的な考え方です。
では、これをどのように実装するのが最適でしょうか？

## メッセージの永続化

[LangGraph](https://langchain-ai.github.io/langgraph/) は組み込みの永続化レイヤーを実装しており、複数の会話ターンをサポートするチャットアプリケーションに最適です。

チャットモデルを最小限の LangGraph アプリケーションでラップすることで、メッセージ履歴を自動的に永続化できるため、マルチターンアプリケーションの開発が簡素化されます。

LangGraph にはシンプルなメモリ内チェックポインタが付属しており、以下でこれを使用します。詳細については、[ドキュメント](https://langchain-ai.github.io/langgraph/concepts/persistence/) を参照してください。これには、SQLite や Postgres などのさまざまな永続化バックエンドの使用方法も含まれます。

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

次に、毎回実行可能ファイルに渡す `config` を作成する必要があります。この config には、入力ファイルに直接含まれない情報が含まれていますが、それでも役立ちます。今回の場合は、`thread_id` を含めます。これは以下のようになります:

In [7]:
config = {"configurable": {"thread_id": "abc123"}}

これにより、単一のアプリケーションで複数の会話スレッドをサポートできるようになります。これは、アプリケーションに複数のユーザーがいる場合によくある要件です。

次に、アプリケーションを起動します:

In [8]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today, Bob?


素晴らしい！チャットボットが私たちに関する情報を記憶するようになりました。別の `thread_id` を参照するように設定を変更すると、会話が新しく開始されることがわかります。

In [10]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?


ただし、元の会話にいつでも戻ることができます（データベースに保存されているため）。

In [11]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. What would you like to discuss today?


このように、多くのユーザーと会話するチャットボットをサポートできます。

:::tip

非同期サポートの場合は、`call_model` ノードを非同期関数に更新し、アプリケーションを呼び出すときに `.ainvoke` を使用します。

```python
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
```

:::

現時点では、モデルの周りにシンプルな永続化レイヤーを追加しただけです。プロンプトテンプレートを追加することで、チャットボットをより複雑でパーソナライズされたものにすることができます。

## プロンプトテンプレート

[プロンプトテンプレート](/docs/concepts/prompt_templates) は、生のユーザー情報を LLM が処理できる形式に変換するのに役立ちます。この場合、生のユーザー入力は単なるメッセージであり、LLM に渡されます。これをもう少し複雑にしてみましょう。まず、カスタム指示を含むシステムメッセージを追加します（ただし、メッセージは引き続き入力として受け取ります）。次に、メッセージ以外の入力も追加します。

システムメッセージを追加するには、`ChatPromptTemplate` を作成します。`MessagesPlaceholder` を使用してすべてのメッセージを渡します。

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

これで、このテンプレートを組み込むようにアプリケーションを更新できます:

In [13]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    # highlight-start
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    # highlight-end
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

同じ方法でアプリケーションを呼び出します:

In [14]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! What brings ye to these waters today? Be ye seekin' treasure, knowledge, or perhaps a good tale from the high seas? Arrr!


In [15]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ye be called Jim, matey! A fine name fer a swashbuckler such as yerself! What else can I do fer ye? Arrr!


素晴らしいですね！では、プロンプトをもう少し複雑にしてみましょう。プロンプトテンプレートが以下のようになっていると仮定しましょう:

In [19]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

プロンプトに新しい `language` 入力を追加したことに注目してください。アプリケーションには現在、入力の `messages` と `language` という 2 つのパラメータがあります。これを反映するために、アプリケーションの状態を更新する必要があります。

In [20]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# highlight-next-line
class State(TypedDict):
    # highlight-next-line
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # highlight-next-line
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    # highlight-next-line
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo puedo ayudarte hoy?


状態全体が永続化されるため、変更が不要な場合は `language` などのパラメータを省略できることに注意してください。

In [22]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. ¿Hay algo más en lo que pueda ayudarte?


内部で何が起こっているかを理解するには、[この LangSmith トレース](https://smith.langchain.com/public/15bd8589-005c-4812-b9b9-23e74ba4c3c6/r) を確認してください。

## 会話履歴の管理

チャットボットを構築する際に理解しておくべき重要な概念の一つは、会話履歴の管理方法です。管理を行わないと、メッセージリストが無制限に大きくなり、LLMのコンテキストウィンドウをオーバーフローする可能性があります。そのため、渡すメッセージのサイズを制限するステップを追加することが重要です。

**重要なのは、この処理はプロンプトテンプレートの前、かつメッセージ履歴から以前のメッセージを読み込んだ後に行う必要があるということです。**

これを実現するには、プロンプトの前に `messages` キーを適切に変更する簡単なステップを追加し、その新しいチェーンをメッセージ履歴クラスでラップします。

LangChainには、[メッセージリストの管理](/docs/how_to/#messages)用のヘルパーがいくつか組み込まれています。今回は、[trim_messages](/docs/how_to/trim_messages/)ヘルパーを使用して、モデルに送信するメッセージの数を減らします。トリマーを使用すると、保持するトークンの数を指定できるほか、システム メッセージを常に保持するかどうかや部分的なメッセージを許可するかどうかなどの他のパラメータも指定できます。

In [23]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

これをチェーン内で使用するには、プロンプトに `messages` 入力を渡す前にトリマーを実行する必要があります。

In [24]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    # highlight-start
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    # highlight-end
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

ここでモデルに名前を尋ねても、チャット履歴のその部分をトリミングしたため、モデルは名前を認識できません。

In [25]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

# highlight-next-line
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. You haven't told me yet!


しかし、最後の数個のメッセージ内の情報について尋ねると、次のことが記憶されます:

In [26]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 equals.


LangSmith を見れば、[LangSmith トレース](https://smith.langchain.com/public/04402eaa-29e6-4bb1-aa91-885b730b6c21/r) で内部で何が起こっているかを正確に確認できます。

## ストリーミング

これでチャットボットが機能するようになりました。しかし、チャットボットアプリケーションにおいて*本当に*重要なUXの考慮事項の一つはストリーミングです。LLMは応答に時間がかかることがあるため、ユーザーエクスペリエンスを向上させるために、多くのアプリケーションでは、生成されたトークンをストリーミングで返しています。これにより、ユーザーは進行状況を確認できます。

これは実はとても簡単です！

デフォルトでは、LangGraphアプリケーションの`.stream`はアプリケーションのステップ（この場合は、モデルレスポンスの1ステップ）をストリーミングします。`stream_mode="messages"`を設定すると、出力トークンをストリーミングできます。

In [27]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
# highlight-next-line
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    # highlight-next-line
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|’s| a| joke| for| you|:

|Why| don|’t| skeleton|s| fight| each| other|?

|Because| they| don|’t| have| the| guts|!||

## 次のステップ

LangChain でチャットボットを作成する方法の基本を理解したら、より高度なチュートリアルもご覧ください。

- [会話型 RAG](/docs/tutorials/qa_chat_history): 外部データソースを介してチャットボットエクスペリエンスを実現します。
- [エージェント](/docs/tutorials/agents): アクションを実行できるチャットボットを構築します。

さらに詳しい内容を知りたい場合は、以下のチュートリアルもご覧ください。

- [ストリーミング](/docs/how_to/streaming): ストリーミングはチャットアプリケーションにとって*極めて重要*です。
- [メッセージ履歴の追加方法](/docs/how_to/message_history): メッセージ履歴に関するあらゆる詳細について解説します。
- [大量のメッセージ履歴の管理方法](/docs/how_to/trim_messages/): 大量のチャット履歴を管理するためのその他のテクニック
- [LangGraph メインdocs](https://langchain-ai.github.io/langgraph/): LangGraphを使ったビルドの詳細については、